# Yfinance_Usecase 
## developing python definition funneled into langchain tool thus to be selected  

In [1]:
from typing import Type
from langchain.llms import OpenAI
from langchain.tools import BaseTool
from pydantic import BaseModel, Field

from datetime import datetime, timedelta

import yfinance as yf

In [13]:
def get_current_stock_price(ticker):
    """Method to get current stock price"""

    ticker_data = yf.Ticker(ticker)
    recent = ticker_data.history(period="1mo")
    return {"price": recent.iloc[0]["Close"], \
            #"currency": ticker_data.info["currency"]
            }


def get_stock_performance(ticker, days):
    """Method to get stock price change in percentage"""

    past_date = datetime.today() - timedelta(days=days)
    ticker_data = yf.Ticker(ticker)
    history = ticker_data.history(start=past_date)
    old_price = history.iloc[0]["Close"]
    current_price = history.iloc[-1]["Close"]
    return {"percent_change": ((current_price - old_price) / old_price) * 100}

class CurrentStockPriceInput(BaseModel):
    """Inputs for get_current_stock_price"""

    ticker: str = Field(description="Ticker symbol of the stock")


class CurrentStockPriceTool(BaseTool):
    name = "get_current_stock_price"
    description = """
        Useful when you want to get current stock price.
        You should enter the stock ticker symbol recognized by the yahoo finance
        """
    args_schema: Type[BaseModel] = CurrentStockPriceInput

    def _run(self, ticker: str):
        price_response = get_current_stock_price(ticker)
        return price_response

    def _arun(self, ticker: str):
        raise NotImplementedError("get_current_stock_price does not support async")



In [18]:
class StockPerformanceInput(BaseModel):
    """Inputs for get_stock_performance"""

    ticker: str = Field(description="Ticker symbol of the stock")
    days: int = Field(description="days transacted over the market")


class StockPerformanceTool(BaseTool):
    name = "get_stock_performance"
    description = """
        Useful when you want to get the stock performance over the given period.
        You should enter the stock ticker symbol recognized by the yahoo finance
        """
    args_schema: Type[BaseModel] = StockPerformanceInput

    def _run(self, ticker: str , days: int):
        performance_response = get_stock_performance(ticker,days)
        return performance_response

    def _arun(self, ticker: str):
        raise NotImplementedError("get_stock_performance does not support async")

In [4]:
from langchain.agents import AgentType, initialize_agent
from langchain.chat_models import ChatOpenAI
from langchain.llms import OpenAI


In [19]:
llm = ChatOpenAI(model="gpt-3.5-turbo-0613", temperature=0, openai_api_key="")

tools = [CurrentStockPriceTool(),StockPerformanceTool()]

agent = initialize_agent(tools, llm, agent=AgentType.OPENAI_FU  NCTIONS, verbose=True)

agent.run("get me the stock price of Samsung Electronics and how did it perform over the past 5 years?")



> Entering new AgentExecutor chain...

Invoking: `get_current_stock_price` with `{'ticker': '005930.KS'}`


{'price': 70900.0}
Invoking: `get_stock_performance` with `{'ticker': '005930.KS', 'days': 1825}`


{'percent_change': 103.63132730006008}The current stock price of Samsung Electronics is 70,900 KRW. 

Over the past 5 years, the stock has performed very well with a percent change of 103.63%.

> Finished chain.


'The current stock price of Samsung Electronics is 70,900 KRW. \n\nOver the past 5 years, the stock has performed very well with a percent change of 103.63%.'